In [1]:
import pandas as pd

In [11]:
df=pd.read_csv("data/Dataset_v3.csv",encoding="latin1",sep=";")
df=df[['id deudor', 'Fecha carga DL', 'operation_id', 'amount',
       'payment_method', 'anual_sales_volume', 'mean_time', 'invoice_mean_val',
       'entity_type', 'relationship_years', 'creation_date', 'payment_date',
       'city', 'industry', 'result']]

In [12]:
df.head()

,id deudor,Fecha carga DL,operation_id,amount,payment_method,anual_sales_volume,mean_time,invoice_mean_val,entity_type,relationship_years,creation_date,payment_date,city,industry,result
0,1.0,4/6/2021,1.0,"18714,00",'cheque',391000.0,90 días,Entre USD 5000 y USD 20000,persona natural,5.0,25/5/2021,25/5/2021,NaN,9.0,Aceptada
1,2.0,2/8/2022,2.0,"1596,00",'cheque',33000.0,60 días,Menos de USD 1000,persona jurídica,7.0,23/7/2022,2/12/2022,11.0,12.0,Aceptada
2,2.0,28/8/2022,3.0,"19152,00",'cheque',409000.0,30 días,Entre USD 20000 y USD 100000,persona natural,1.0,18/8/2022,16/12/2022,12.0,10.0,Aceptada
3,4.0,16/7/2022,4.0,"6354,00",'cheque',68000.0,90 días,Entre USD 5000 y USD 20000,persona jurídica,8.0,6/7/2022,29/9/2022,17.0,5.0,Aceptada
4,5.0,27/5/2022,5.0,"9729,72",'cheque',214000.0,120 días,Entre USD 20000 y USD 100000,persona natural,3.0,17/5/2022,30/8/2022,7.0,15.0,Aceptada


In [ ]:


def conteo_operaciones(Data):

    return pass

In [1]:
import boto3

client = boto3.client('athena')

response = client.create_named_query(
    Name='TestQuery',
    Database='wbc',
    region=
    QueryString='SELECT * FROM "wbc"."thirdpartydata" limit 10;',
)
print(response)

NoRegionError: You must specify a region.

In [2]:
import awswrangler as wr  
import boto3

client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN
)


boto3.setup_default_session(region_name="us-east-1")
df = wr.athena.read_sql_query(sql="SELECT * FROM dataset_v4", database="facturedo")

NoRegionError: You must specify a region.

In [1]:
import boto3
from Consultor import AWS_Model
from config import session_dict

In [5]:
input1={"client_id":1}
api=AWS_Model(session_dict,input=input1)
df=api.bd_to_dataframe()
df.head()

Querying AWS Athena...
Query completed, data retrieved successfully!


,id_deudor,fecha_carga_dl,operation_id,amount,payment_method,anual_sales_volume,mean_time,invoice_mean_val,entity_type,relationship_years,creation_date,payment_date,city,industry,result
0,1,4/6/2021,1,18714.000000,'cheque',391000.0,90 d�as,Entre USD 5000 y USD 20000,persona natural,5,25/5/2021,25/5/2021,<NA>,9,Aceptada
1,1,17/4/2022,9,17337.880859,'transferencia bancaria',426000.0,120 d�as,Entre USD 1000 y USD 5000,persona natural,1,7/4/2022,20/7/2022,8,13,Rechazada
2,1,24/6/2021,72,14974.000000,'transferencia bancaria',368000.0,90 d�as,Entre USD 1000 y USD 5000,persona natural,1,14/6/2021,25/8/2021,12,6,Rechazada
3,1,2/1/2022,129,6736.279785,'transferencia bancaria',110000.0,30 d�as,Entre USD 1000 y USD 5000,persona natural,1,23/12/2021,11/7/2022,20,10,Rechazada
4,1,2/1/2022,164,7424.000000,'cheque',98000.0,120 d�as,Entre USD 1000 y USD 5000,persona natural,1,23/12/2021,12/4/2022,11,10,Aceptada


In [31]:
def conteo_operaciones(df):
    """
    Agrupar por cada valor de la columna result y contar el número de operaciones
    esta variable debe retornar un diccionario, similar a {Aceptada: 20, Rechazada: 15}.
    """
    return df.groupby(["result"])["result"].count().to_dict()


def dias_promedio_pago_ops_pagadas(df):
    """
    Agrupar las operaciones con la etiqueta Pagada y retornar el promedio de días. El cálculo de días se realiza restando la columna creation_date y payment date.
    """

    df_temp=df[df["result"]=="Pagada"].copy()    
    df_temp["creation_date2"]=pd.to_datetime(df_temp["creation_date"],format='%d/%m/%Y')
    df_temp["payment_date2"]=pd.to_datetime(df_temp["payment_date"],format='%d/%m/%Y')
    df_temp["diff"]=df_temp["payment_date2"]-df_temp["creation_date2"]
    df_temp["diff"]=df_temp["diff"].dt.days
    m=int(df_temp["diff"].mean())

    return m


def umbrales_outliers(df):
    """
    Umbral mínimo y máximo de la columna amount, para identificar outliers 

    """

    stats=df["amount"].describe()
    Q3=stats["75%"]
    Q1=stats["25%"]
    L=Q1-1.5*(Q3-Q1)
    U=Q1+1.5*(Q3-Q1)
    dict_LU={"umbral_inf": L, "umbral_superior": U}
    return dict_LU

def tipo_pago_max_volumen(df):

    """
    Máximo volumen, agrupado por payment_method
    """

    return df[["payment_method","amount"]].groupby(["payment_method"]).max()["amount"].to_dict()

def lista_operaciones(df):
    """
    Lista de operaciones ubicadas en el umbral superior,
    """

    dict_LU=umbrales_outliers(df)
    df_temp=df[["operation_id","amount"]][df["amount"]>dict_LU["umbral_superior"]]
    result=dict(zip(df_temp.operation_id, df_temp.amount))
    return result
    


In [21]:
import pandas as pd
df["creation_date2"]=pd.to_datetime(df["creation_date"],format='%d/%m/%Y')
df["payment_date2"]=pd.to_datetime(df["payment_date"],format='%d/%m/%Y')
df["diff"]=df["payment_date2"]-df["creation_date2"]
df["diff"]=df["diff"].dt.days
m=int(df["diff"].mean())
m

142

In [30]:
stats=df["amount"].describe()
Q3=stats["75%"]
Q1=stats["25%"]
L=Q1-1.5*(Q3-Q1)
U=Q1+1.5*(Q3-Q1)
print(L,U)

-8672.0341796875 23520.0341796875


In [34]:
df[["payment_method","amount"]].groupby(["payment_method"]).max()["amount"].to_dict()

{"'cheque'": 18714.0, "'transferencia bancaria'": 18154.689453125}

In [39]:
df_temp=df[["operation_id","amount"]][df["amount"]>10000]
result=dict(zip(df_temp.operation_id, df_temp.amount))
result

{'1': 18714.0,
 '9': 17337.880859375,
 '72': 14974.0,
 '188': 14858.0,
 '195': 15556.98046875,
 '278': 18154.689453125}